In [1]:
import pandas as pd

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Alejandra\anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-7dd3504c366f>", line 1, in <module>
    import pandas as pd
  File "C:\Users\Alejandra\anaconda3\envs\tf\lib\site-packages\pandas\__init__.py", line 11, in <module>
    __import__(dependency)
  File "C:\Users\Alejandra\anaconda3\envs\tf\lib\site-packages\numpy\__init__.py", line 145, in <module>
    from . import lib
  File "C:\Users\Alejandra\anaconda3\envs\tf\lib\site-packages\numpy\lib\__init__.py", line 39, in <module>
    from .npyio import *
  File "<frozen importlib._bootstrap>", line 983, in _find_and_load
  File "<frozen importlib._bootstrap>", line 967, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 677, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib.

KeyboardInterrupt: 

In [ ]:
import pathlib

# define the path
directory = pathlib.Path('Interpolation')

# define the pattern
currentCompany = 'RPD'

for currentFile in directory.glob(currentCompany):
    print(currentFile)

fileDen = 'InterpolatedDenWeek' + str(currentCompany) + '.csv'
modelDen = 'modelDen' + str(currentCompany) + '.json'
weightsDen = 'modelDen' + str(currentCompany) + '.h5'
r2ScoreDen = str(currentCompany) + '_R2ScoreDen' +  '.txt'

print(currentFile/fileDen)
print(currentFile/modelDen)
print(currentFile/weightsDen)
print(currentFile/r2ScoreDen)

In [ ]:
df = pd.read_csv(currentFile/fileDen)

In [ ]:
df

In [ ]:
df["D FCF"].mean()

In [ ]:
dataset = df.values

In [ ]:
dataset

In [ ]:
X = dataset[:,1:7]
Y = dataset[:,7]

In [ ]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)


In [ ]:
X_scale

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X_scale, Y, test_size=0.2)
X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)
print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
model = Sequential([
    Dense(6, activation='elu', input_shape=(6,)),
    Dense(64, activation='elu'),
    Dense(64, activation='elu'),
    Dense(64, activation='elu'),
    Dense(64, activation='elu'),
    Dense(64, activation='elu'),
    Dense(64, activation='elu'),
    Dense(1, activation='elu'),
])

In [ ]:
model.compile(optimizer='rmsprop',
              loss='mean_absolute_error')

In [ ]:
hist = model.fit(X_train, Y_train,
          batch_size=2, epochs=1500,
          validation_data=(X_val, Y_val))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open(currentFile/modelDen, "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(currentFile/weightsDen)
print("Saved model to disk")

In [ ]:
import numpy as np
from numpy import array
arr = []
for x in range(len(X_scale)):
    Xnew = array([[X_scale[x][0], X_scale[x][1] , X_scale[x][2], X_scale[x][3], X_scale[x][4], X_scale[x][5]]])
    ynew = model.predict(Xnew)
    #print(ynew[0][0])
    arr.append(ynew[0][0])
array = np.array(arr)

In [ ]:
from sklearn.metrics import r2_score
r2_score(Y.tolist(), array.tolist())

In [ ]:
myText = open(currentFile/r2ScoreDen,'w')
myString = str(r2_score(Y.tolist(), array.tolist()))
myText.write(myString)
myText.close()